In [1]:
!pip install numpy<2
!pip install rdkit-pypi -q

/bin/bash: line 1: 2: No such file or directory
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 55.8 MB/s eta 0:00:00


In [3]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Descriptors
from rdkit.Chem import rdMolDescriptors
from rdkit.Chem import rdFreeSASA
from rdkit.Chem.rdForceFieldHelpers import UFFOptimizeMolecule
import numpy as np
from google.colab import drive
import time

In [4]:
drive.mount('/content/drive')


train_df_path = '/content/drive/MyDrive/dacon_1/train.csv'
candidates_df_path = '/content/drive/MyDrive/dacon_1/candidates_03_1.csv'

output_candidates_path = '/content/drive/MyDrive/dacon_1/candidates_05_0.csv'

train_df = pd.read_csv(train_df_path)
candidates_df = pd.read_csv(candidates_df_path)


Mounted at /content/drive


In [ ]:
def smiles_to_mol(smiles):
    try:
        return Chem.MolFromSmiles(smiles)
    except:
        return None

train_df['Mol'] = train_df['Canonical_Smiles'].apply(smiles_to_mol)

# conformer
def calculate_3d_features_from_conformer(mol):
    if mol is None:
        return {
            'PMI1': None, 'PMI2': None, 'PMI3': None,
            'NPR1': None, 'NPR2': None,
            'SpherocityIndex': None,
            'SASA': None, 'MolVolume': None,
            'SlogP_VSA1': None, 'SlogP_VSA2': None, 'SlogP_VSA3': None,
            'SlogP_VSA4': None, 'SlogP_VSA5': None, 'SlogP_VSA6': None,
            'SlogP_VSA7': None, 'SlogP_VSA8': None, 'SlogP_VSA9': None,
            'SlogP_VSA10': None, 'SlogP_VSA11': None,
            'PEOE_VSA1': None, 'PEOE_VSA2': None, 'PEOE_VSA3': None,
            'PEOE_VSA4': None, 'PEOE_VSA5': None, 'PEOE_VSA6': None,
            'PEOE_VSA7': None, 'PEOE_VSA8': None, 'PEOE_VSA9': None,
            'PEOE_VSA10': None, 'PEOE_VSA11': None, 'PEOE_VSA12': None,
            'PEOE_VSA13': None, 'PEOE_VSA14': None
        }

    mol_with_hs = Chem.AddHs(mol)

    num_conformers = 20  # 생성할 컨포머 개수 (많을수록 정확도, 시간 증가)
    max_opt_iters = 500  # 최적화 반복 횟수
    prune_rms_thresh = 0.5 # 0.5 Å 미만 RMSD는 중복으로 간주
    random_seed = 42


    params = AllChem.ETKDGv2()
    params.randomSeed = random_seed
    params.pruneRmsThresh = prune_rms_thresh

    try:
        cids = AllChem.EmbedMultipleConfs(mol_with_hs, numConfs=num_conformers, params=params)
        if not cids:
            raise ValueError("No conformers generated.")
    except Exception as e:
        return {k: None for k in ['PMI1', 'PMI2', 'PMI3', 'NPR1', 'NPR2', 'SpherocityIndex',
                                  'SASA', 'MolVolume',
                                  'SlogP_VSA1', 'SlogP_VSA2', 'SlogP_VSA3', 'SlogP_VSA4',
                                  'SlogP_VSA5', 'SlogP_VSA6', 'SlogP_VSA7', 'SlogP_VSA8',
                                  'SlogP_VSA9', 'SlogP_VSA10', 'SlogP_VSA11',
                                  'PEOE_VSA1', 'PEOE_VSA2', 'PEOE_VSA3', 'PEOE_VSA4',
                                  'PEOE_VSA5', 'PEOE_VSA6', 'PEOE_VSA7', 'PEOE_VSA8',
                                  'PEOE_VSA9', 'PEOE_VSA10', 'PEOE_VSA11', 'PEOE_VSA12',
                                  'PEOE_VSA13', 'PEOE_VSA14']}

    min_energy = float('inf')
    best_conf_id = -1
    for cid in cids:
        try:
            status = AllChem.UFFOptimizeMolecule(mol_with_hs, confId=cid, maxIters=max_opt_iters)
            energy = AllChem.UFFGetMoleculeForceField(mol_with_hs, confId=cid).CalcEnergy()
            if energy < min_energy:
                min_energy = energy
                best_conf_id = cid
        except Exception as e:
            continue

    if best_conf_id == -1:
        return {k: None for k in ['PMI1', 'PMI2', 'PMI3', 'NPR1', 'NPR2', 'SpherocityIndex',
                                  'SASA', 'MolVolume',
                                  'SlogP_VSA1', 'SlogP_VSA2', 'SlogP_VSA3', 'SlogP_VSA4',
                                  'SlogP_VSA5', 'SlogP_VSA6', 'SlogP_VSA7', 'SlogP_VSA8',
                                  'SlogP_VSA9', 'SlogP_VSA10', 'SlogP_VSA11',
                                  'PEOE_VSA1', 'PEOE_VSA2', 'PEOE_VSA3', 'PEOE_VSA4',
                                  'PEOE_VSA5', 'PEOE_VSA6', 'PEOE_VSA7', 'PEOE_VSA8',
                                  'PEOE_VSA9', 'PEOE_VSA10', 'PEOE_VSA11', 'PEOE_VSA12',
                                  'PEOE_VSA13', 'PEOE_VSA14']}

    features = {}

    try:
        features['PMI1'] = rdMolDescriptors.CalcPMI1(mol_with_hs, confId=best_conf_id)
        features['PMI2'] = rdMolDescriptors.CalcPMI2(mol_with_hs, confId=best_conf_id)
        features['PMI3'] = rdMolDescriptors.CalcPMI3(mol_with_hs, confId=best_conf_id)
        features['NPR1'] = rdMolDescriptors.CalcNPR1(mol_with_hs, confId=best_conf_id)
        features['NPR2'] = rdMolDescriptors.CalcNPR2(mol_with_hs, confId=best_conf_id)
    except Exception:
        features['PMI1'] = None
        features['PMI2'] = None
        features['PMI3'] = None
        features['NPR1'] = None
        features['NPR2'] = None

    try:
        features['SpherocityIndex'] = Descriptors.SpherocityIndex(mol_with_hs, confId=best_conf_id)
    except Exception:
        features['SpherocityIndex'] = None

    try:
        ptable = Chem.GetPeriodicTable()
        radii = [ptable.GetRvdw(atom.GetAtomicNum()) for atom in mol_with_hs.GetAtoms()]
        features['SASA'] = rdFreeSASA.CalcSASA(mol_with_hs, radii, confIdx=best_conf_id)
    except Exception:
        features['SASA'] = None

    try:
        features['MolVolume'] = Descriptors.MolVolume(mol_with_hs, confId=best_conf_id)
    except Exception:
        features['MolVolume'] = None

    for i in range(1, 12): # SlogP_VSA1 ~ SlogP_VSA11
        try:
            features[f'SlogP_VSA{i}'] = getattr(Descriptors, f'SlogP_VSA{i}')(mol_with_hs)
        except Exception:
            features[f'SlogP_VSA{i}'] = None
    for i in range(1, 15): # PEOE_VSA1 ~ PEOE_VSA14
        try:
            features[f'PEOE_VSA{i}'] = getattr(Descriptors, f'PEOE_VSA{i}')(mol_with_hs)
        except Exception:
            features[f'PEOE_VSA{i}'] = None

    return features

print("3D 피처 계산 시작...")
start_time = time.time()

rdkit_3d_features_df = train_df['Mol'].apply(calculate_3d_features_from_conformer).apply(pd.Series)

end_time = time.time()
print(f" {end_time - start_time:.2f} s")

train_df = train_df.drop(columns=['Mol'])

train_df_with_3d_rdkit = pd.concat([train_df, rdkit_3d_features_df], axis=1)

_3d_feature_names = list(rdkit_3d_features_df.columns)
features_to_merge_into_candidates = train_df_with_3d_rdkit[['ID'] + _3d_feature_names].set_index('ID')


print("merging...")
candidates_03_1_with_3d_rdkit_features = pd.merge(
    candidates_df,
    features_to_merge_into_candidates,
    on='ID',
    how='left'
)


print("saving...")
candidates_00_final = candidates_03_1_with_3d_rdkit_features
train_00_final = train_df_with_3d_rdkit

candidates_00_final.to_csv(output_candidates_path, index=False)

3D 피처 계산 시작...
